In [1]:
# https://quantnomad.com/2020/12/03/getting-historical-bars-from-bybit-api-with-python/

# https://bybit-exchange.github.io/docs/linear/#t-querykline

import requests 
import json 
import pandas as pd
import datetime as dt
from datetime import date
import matplotlib.pyplot as plt
import time
import pytz; 

import os
from dotenv import load_dotenv

import plotly.graph_objects as go
import openpyxl
from openpyxl import load_workbook

### Set environment variables from the .env in the local environment

In [2]:
load_dotenv()
my_api_key = os.getenv("API_KEY")
my_api_secret = os.getenv("API_SECRET")

# print(my_api_key)
# print(my_api_secret)

### Back Testing Parameters

In [3]:
SYMBOL = 'BTCUSDT'
FROM_TIME = dt.datetime(2021, 10, 1) # (year, month, day, hour, minutes, seconds)
TO_TIME = dt.datetime(2021, 11, 26)
TIMEFRAME = 720 # Possible Values: 1 3 5 15 30 60 120 240 360 720 "D" "W" "M"

### Change Timezone if Needed

In [3]:
## my_list simply unpacks the list elements and pass each one of them as parameters to the print function  
# print(*pytz.all_timezones, sep='\n')

# In windows command prompt try:
#     This gives current timezone: tzutil /g
#     This gives a list of timezones: tzutil /l
#     This will set the timezone: tzutil /s "Central America Standard Time"


#os.system('tzutil /s "Eastern Standard Time"')  
#os.system('tzutil /s "Singapore Standard Time"')
#time.strftime('%Y-%m-%d %H:%m') 

### Create ByBit Session

In [4]:
# https://github.com/verata-veritatis/pybit

# Import pybit and define the HTTP object.
from pybit import HTTP

"""
You can create an authenticated or unauthenticated HTTP session. 
You can skip authentication by not passing any value for api_key
and api_secret.
"""

# Unauthenticated
#session_unauth = HTTP(endpoint='https://api.bybit.com')

# Authenticated
session_auth = HTTP(
    endpoint = 'https://api.bybit.com',
    api_key = my_api_key,
    api_secret = my_api_secret
)

# Lets get market information about EOSUSD. Note that 'symbol' is
# a required parameter as per the Bybit API documentation.
#session_unauth.latest_information_for_symbol(symbol='BTCUSD')

# We can fetch our wallet balance using an auth'd session.
#session_auth.get_wallet_balance(coin='BTC')


## Spot API.

# Unauthenticated, prefer spot endpoint
#session_spot_unauth = HTTP(endpoint='https://api.bybit.com', spot=True)

# Prefer spot endpoint for this request, without specifying at obj creation
session_auth.get_wallet_balance(coin='BTC', spot=True)

{'ret_code': 0,
 'ret_msg': '',
 'ext_code': None,
 'ext_info': None,
 'result': {'balances': [{'coin': 'BTC',
    'coinId': 'BTC',
    'coinName': 'BTC',
    'total': '0.00012422',
    'free': '0.00012422',
    'locked': '0'}]}}

In [5]:
# query_kline parameters
# ----------------------------------------------------------------------
# Name      Required    Type     Description
# ----      --------    ----     -----------
# symbol	true	    string	 Symbol
# interval	true	    string	 Data refresh interval. Enum : 1 3 5 15 30 60 120 240 360 720 "D" "M" "W"
# from	    true	    integer	 From timestamp in seconds
# limit     false       integer  Limit for data size per page, max size is 200. Default as showing 200 pieces of data per page
# ----------------------------------------------------------------------

def get_bybit_bars(symbol, interval, fromTime):
 
    fromTime = str(int(fromTime.timestamp()))
    
    result = session_auth.query_kline(symbol=symbol, interval=interval, **{'from':fromTime})['result']
    df = pd.DataFrame(result)
 
    if (len(df.index) == 0):
        return None
    
    df.index = [dt.datetime.utcfromtimestamp(x) for x in df.open_time]
 
    return df

# The issue with ByBit API is that you can get a maximum of 200 bars from it. 
# So if you need to get data for a large portion of the time you have to call it multiple times.

def get_bybit_kline_data(symbol, interval, fromTime, toTime):
    
    df_list = []
    last_datetime = fromTime
    while last_datetime < toTime:
        print(f'Fetching next 200 lines fromTime: [{last_datetime}]')
        new_df = get_bybit_bars(symbol, interval, last_datetime)
        if new_df is None:
            break
        df_list.append(new_df)
        last_datetime = max(new_df.index) + dt.timedelta(0, 1) # Add 1s to last data received
        
        #time.sleep(2) # Sleep for x seconds, to avoid being locked out
 
    df = pd.concat(df_list)
    
    # Drop rows that have a timestamp greater than toTime
    df = df[df.open_time <= int(toTime.timestamp())]

    return df

In [7]:
# fetch data from Bybit and save locally into csv /excel files

df = get_bybit_kline_data(SYMBOL, TIMEFRAME, FROM_TIME, TO_TIME)

from_str = FROM_TIME.strftime('%Y-%m-%d')
to_str = TO_TIME.strftime('%Y-%m-%d')

filename_csv = f'ByBit Historical\\{SYMBOL}_{from_str}_to_{to_str}_{TIMEFRAME}.csv'
filename_xlsx = f'ByBit Historical\\{SYMBOL}_{from_str}_to_{to_str}_{TIMEFRAME}.xlsx'

df.to_csv (filename_csv, index = True, header=True)
df.to_excel (filename_xlsx, index = True, header=True)

df

Fetching next 200 lines fromTime: [2021-10-01 00:00:00]


,id,symbol,period,interval,start_at,open_time,volume,open,high,low,close,turnover
2021-10-01 00:00:00,13318601,BTCUSDT,720,720,1633046400,1633046400,47495.393,43757.5,47949.5,43273.0,47486.5,2.255390e+09
2021-10-01 12:00:00,13353514,BTCUSDT,720,720,1633089600,1633089600,31282.228,47486.5,48440.0,46788.0,48092.5,1.504441e+09
2021-10-02 00:00:00,13388420,BTCUSDT,720,720,1633132800,1633132800,13689.434,48092.5,48170.0,47439.0,47670.0,6.525753e+08
2021-10-02 12:00:00,13423325,BTCUSDT,720,720,1633176000,1633176000,15493.378,47670.0,48315.0,47536.5,47650.0,7.382595e+08
2021-10-03 00:00:00,13458265,BTCUSDT,720,720,1633219200,1633219200,13582.108,47650.0,48055.0,47098.0,47959.5,6.513911e+08
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-23 12:00:00,19214179,BTCUSDT,720,720,1637668800,1637668800,13216.333,56139.0,57902.0,56139.0,57586.5,7.561220e+08
2021-11-24 00:00:00,19292435,BTCUSDT,720,720,1637712000,1637712000,11749.254,57586.5,57732.0,56125.0,56683.5,6.670649e+08
2021-11-24 12:00:00,19372513,BTCUSDT,720,720,1637755200,1637755200,11242.059,56683.5,57495.0,55935.0,57185.5,6.372338e+08
2021-11-25 00:00:00,19455601,BTCUSDT,720,720,1637798400,1637798400,12358.142,57185.5,58499.5,57041.5,58035.0,7.138206e+08


### Optional Candlestick Chart

In [13]:
# format_date = lambda d: d.strftime('%Y-%m-%d %H:%M')

# nb_rows = len(df)
# x_step = int(nb_rows / 20)


# fig = go.Figure(data=[go.Candlestick(x=df['open_time'],
#                 open=df['open'],
#                 high=df['high'],
#                 low=df['low'],
#                 close=df['close'])])

# fig.update_layout(
#     title=f'{SYMBOL} Prices',
#     yaxis_title="Price",
#     width=1200, height=800,
#     xaxis_rangeslider_visible=True,
#     xaxis_tickformat = '%Y-%m-%d %H:%M',
#     xaxis = dict(
#         tickmode = 'array',
#         tickvals = df['open_time'][::x_step],
#         ticktext = format_date(df.index[::x_step])
#     )
# )

# fig.show()